In [ ]:
#importing modules
import os
from nltk.corpus import stopwords
import string
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [ ]:
#LIST OF FOLDER NAMES FROM WHICH FILES ARE TO BE EXTRACTED
y=['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [ ]:
#MAKING OF STOP WORDS FROM MODULE NLTK
stop=stopwords.words('english')
punctuations=list(string.punctuation)
stop = stop + punctuations

In [ ]:
#path in my pc of the folder
path='C:\\Users\\Psyfer\\Desktop\\ml\\20_newsgroups'

In [ ]:
#INTIALIZING THE Y_TRUE (WHICH IS ACTUAL VALUES OF Y FOR COMPARING AND SCORES)
y_true=[]

In [ ]:
#MAKING OF VOCABUALARY AND THE Y_TRUE USING OS FUNCTIONS TO CHANGE DIRECTORY AND EXTRACT FILES FROM IT AS FILES ARE NOT
#CURRENT DIRECTORY
vocab={}
for i in range(len(y)):
    seq=os.listdir(path+s+y[i])
    os.chdir(path+s+y[i])
    for f in seq:
        read=open(f)
        y_true.append(i)
        for l in read:
            l=l.strip()
            words=l.split()
            for w in words:
                if w.isalpha():
                    if not w.lower() in stop:
                        if w in vocab:
                            vocab[w]+=1
                        else:
                            vocab[w]=1
            

In [ ]:
#IMPORTING MODULE COUNTER WHICH IS USED TO HAVE MOST FREQUENT WORDS
from collections import Counter
d=Counter(vocab)


In [ ]:
#fEATURES ARE MOST COMMON 3000 WORDS FROM VOCAB
f=d.most_common(3000)


In [ ]:
#AS F IS LIST OF TUPLES AND WE WANT TO HAVE ONLY WORD THAT IS 0TH ENTRY OF EACH TUPLE IN LIST
#THIS IS FINAL VOCAB
v=[i[0] for i in f]

In [ ]:
#INTIALIZING ARRAY TO HAVE WORD COUNT FOR EACH FEATURE IN EACH DOCUMENT
r=[[0 for i in range(3000)] for j in range(len(y_true))]
e=-1

In [ ]:
#THIS IS CREATING A 2D ARRAY IN WHICH THE ENTRIES ARE THE WORD OR FEAURE COUNT IN EACH DOCUMENT
for i in range(len(y)):
    seq=os.listdir(path+s+y[i])
    os.chdir(path+s+y[i])
    
    for f in seq:
        e+=1
        read=open(f)
        for l in read:
            l=l.strip()
            words=l.split()
            for w in words:
                if w in v:
                    r[e][v.index(w)]+=1
    
                

In [ ]:
#CREATING DATAFRAME FROM 2D ARRAY
import pandas as pd
df=pd.DataFrame(r)


In [ ]:
#IMPORTING MULTINOMIAL NAIVE BAYES OF SKLEARN
from sklearn.naive_bayes import MultinomialNB

In [ ]:
#OBJECT OF MULTINOMIAL NAIVE BAYES
clf = MultinomialNB()

In [ ]:
#TRAINING DATA WITH DATAFRAME(df) AND Y_TRUE
clf.fit(df, y_true)

In [ ]:
#THIS IS TRAINING ACCURACY
clf.score(df,y_true)

In [ ]:
#CREATING LIST TO NUMPY ARRAY
y_true=np.array(y_true)

In [ ]:
#THIS IS FIT FUNCTION WHICH CREATES DICTIONARY WHICH WILL BE USED TO PREDICT DATA
def fit(x_train,y_train):
    count={}
    possible_classes=list(set(y_train))
    count['total_data']=len(y_train)
    for current_class in possible_classes:
        x=x_train[y_train==current_class]
        count[current_class]={}
        count[current_class]['total_count']=len(x)
        l=x.sum()
        l=np.array(l)
        
        count[current_class]['total_words']=l.sum()

        for f in range(3000):
            count[current_class][f]=x[:][f].sum()
    return count

In [ ]:
#THIS RETURN PROBABILTY OF EACH CLASS FOR A DOCUMENT WHICH HAS WORD BANK AS X_TEST_VOCAB
def probability_singler(count,x_test_vocab,current_class):
    prob=count[current_class]['total_count']/count['total_data']
    for w in x_test_vocab:
        if w in v:
            we=v.index(w)
            we=(count[current_class][we]+1)/(count[current_class]['total_words'] + len(v) )
            prob=(prob)*(we)
    return prob
    

In [ ]:
#THIS RETURN BEST CLASS FOR A DOCUMENT BY COMPARING PROBABILITY AND INPUT X_TEST_VOCAB WHICH IS WORD BANK IN TEST
#DOCUMENT
def probability(count,x_test_vocab,y_train):
    best_p=-10000
    best_class=-1
    possible_classes=list(set(y_train))
    for current_class in possible_classes:
        prob=probability_singler(count,x_test_vocab,current_class)
        
        if(prob>best_p):
            best_p=prob
            best_class=current_class
    return best_class

In [ ]:
#DICTIONARY COUNT OF FIT FUNCTION
count=fit(df,y_true)

In [ ]:
#THIS IS THE NAMES OF FOLDER CONTAINING RANDOM TEST FILES FROM EACH CLASS
y_test_folder=['test'+str(i) for i in range(20)]

In [ ]:
#THIS IS PATH TO TEST FILES
path='C:\\Users\\Psyfer\\Desktop\\ml\\20_newsgroups'

In [ ]:
#INTIALISING y_predict WHICH IS PREDICTED BY OWN CODE AND Y_TEST_TRUE IS TRUE VALUES OF PREDICTION(AS TESTING FILES ARE
#MADE SUCH WE CAN HAVE TRUE VALUES OF PREDICTION FOR TESTING FILES AND COMPARE RESULT
y_predict=[]
y_test_true=[]


In [ ]:
#INTIALSING 2D ARRAY
e=-1
we=[[0 for i in range(3000)] for j in range(2799)]

In [ ]:
#CREATING Y_PREDICT AND Y_TEST_TRUE FOR TESTING FILES
for i in range(len(y_test_folder)):
    #y_test=[]
    seq=os.listdir(path+s+y_test_folder[i])
    os.chdir(path+s+y_test_folder[i])
    for f in seq:
            e+=1
            l=[]
            read=open(f)
            for line in read:
                line=line.strip()
                words=line.split()
                for w in words:
                    if w.isalpha():
                        if not w.lower() in stop:
                            if w in v:
                                we[e][v.index(w)]+=1
                                
                                l.append(w)
            y_predict.append(probability(count,l,y_true))
            
    
            y_test_true.append(i)
   

In [ ]:
#CREATING 2D ARRAY WHICH IS USED BY SKLEARN NAIVE BAYES TO PREDICT
rt=pd.DataFrame(we)

In [ ]:
#PREDICTING on test files USING SKLEARN NAIVE BAYES 
y_predict_sk=clf.predict(rt)

In [ ]:
#TESTING ACCURACY
clf.score(rt,y_test_true)

In [ ]:
#CLASSIFICATION REPORT FOR TESTING FILES BY OWN CODE
print(classification_report(y_test_true, y_predict))

In [ ]:
#CLASSIFICATION REPORT FOR TESTING FILES BY SKLAEARN naive bayes
print(classification_report(y_test_true, y_predict_sk))